<a href="https://colab.research.google.com/github/mmaruthi/Assignment4/blob/master/Gradca0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import cv2
from keras import backend as K
from skimage import io

Using TensorFlow backend.


In [0]:
def gradcam(inp):
    model = VGG16(weights="imagenet")
    from skimage import io
    #dog = io.imread("https://www.rspcapetinsurance.org.au/rspca/media/images/hero/dog-insurance-hero.jpg")
    cat = io.imread(inp)
    cat = cv2.resize(cat, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
    x = image.img_to_array(cat)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    class_idx = np.argmax(preds[0])
    print(class_idx)
    class_output = model.output[:, class_idx]
    last_conv_layer = model.get_layer("block5_conv3")

    grads = K.gradients(class_output, last_conv_layer.output)[0]
    print(grads.shape)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    print(pooled_grads.shape)
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

    pooled_grads_value, conv_layer_output_value = iterate([x])

    for i in range(512):
       conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    heatmap = np.mean(conv_layer_output_value, axis = -1)
    print(conv_layer_output_value.shape)
    print(heatmap.shape)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, (cat.shape[1], cat.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(cat, 0.5, heatmap, 0.5, 0)
    from google.colab.patches import cv2_imshow
    cv2_imshow( cat)
    cv2_imshow(superimposed_img)

In [15]:
gradcam("https://unsplash.com/photos/99wAlgGExpI")

ValueError: ignored